In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import re
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import optuna
from xgboost import XGBRegressor
import joblib
from sklearn.multioutput import MultiOutputRegressor
import ast
import joblib
import matplotlib.pyplot as plt
import random
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
path = "/Users/amrtamer/Documents/Internship"
# path = "/content/drive/MyDrive/Internship"

In [3]:
data = pd.read_pickle(path + r'/data/Volve/Volve_cleaned_prepared.pkl')
if "Unnamed: 0" in data.columns:
    data = data.drop(columns="Unnamed: 0")
data.head()

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,linear_regressor,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029309,0.605973,0.396165,-2.015211,-0.533280,3.281836,...,-0.489929,-0.495877,-0.498713,-0.502834,-0.001433,-0.0027,-0.005271,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845793,0.634777,0.239816,0.088574,-0.419656,2.849981,...,-0.489929,-0.363747,-0.426301,-0.464667,-0.001433,-0.0027,-0.005271,"[0.0, 0.0, -0.4905010756171413]","[0.0, 0.0, 0.0, 0.0, -0.4905010756171413]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775023,0.639809,0.199889,-0.687843,-0.402874,2.607708,...,-0.146007,-0.180982,-0.315168,-0.402787,-0.001433,-0.0027,-0.005271,"[0.0, -0.4905010756171413, -0.09810136648612545]","[0.0, 0.0, 0.0, -0.4905010756171413, -0.098101...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.49..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705031,0.642083,0.161491,1.145565,-0.379061,2.364038,...,0.024781,-0.133505,-0.264997,-0.368282,-0.001433,-0.0027,-0.005271,"[-0.4905010756171413, -0.09810136648612545, 0....","[0.0, 0.0, -0.4905010756171413, -0.09810136648...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.4905010..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625355,0.643889,0.117219,1.145565,-0.340115,2.088747,...,0.057726,-0.077878,-0.210807,-0.330137,-0.084172,-0.0027,-0.005271,"[-0.09810136648612545, 0.18307478352672518, -0...","[0.0, -0.4905010756171413, -0.0981013664861254...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.490501075617..."


In [4]:
well_info = dict(set([(j, i) for i, j in data[['WELL_BORE_CODE', 'well_name']].values.tolist()])) # creating a dictionary of well identiification with {code: name} structure
well_info

{5769: 'NO 15/9-F-5 AH',
 5599: 'NO 15/9-F-12 H',
 7405: 'NO 15/9-F-1 C',
 7078: 'NO 15/9-F-11 H',
 5351: 'NO 15/9-F-14 H',
 7289: 'NO 15/9-F-15 D'}

In [5]:
identification_column = ['well_name', 'WELL_BORE_CODE', 'date']
single_feature_columns = ['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']
multiple_feature_columns = ['Lag_short', 'Lag_medium', 'Lag_long']
feature_columns = single_feature_columns + multiple_feature_columns
target_column = ['oil_rate']

In [6]:
data = data[identification_column + feature_columns]
data.shape

(7987, 26)

In [7]:
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].mean().round(4))
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].std().round(4))

prod_hrs                -0.0
bhp                      0.0
bht                     -0.0
dp_tubing                0.0
AVG_ANNULUS_PRESS        0.0
AVG_CHOKE_SIZE_P        -0.0
whp                     -0.0
wht                     -0.0
choke_size_percentage    0.0
oil_vol                  0.0
gas_vol                 -0.0
water_vol                0.0
linear_regressor         0.0
EMA_short                0.0
EMA_medium              -0.0
EMA_long                 0.0
Rolling_short            0.0
Rolling_medium           0.0
Rolling_long            -0.0
oil_rate                -0.0
dtype: float64
prod_hrs                 1.0
bhp                      1.0
bht                      1.0
dp_tubing                1.0
AVG_ANNULUS_PRESS        1.0
AVG_CHOKE_SIZE_P         1.0
whp                      1.0
wht                      1.0
choke_size_percentage    1.0
oil_vol                  1.0
gas_vol                  1.0
water_vol                1.0
linear_regressor         1.0
EMA_short                1.0

In [8]:
data

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,oil_rate,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029309,0.605973,0.396165,-2.015211,-0.533280,3.281836,...,-0.495877,-0.498713,-0.502834,-0.001433,-0.0027,-0.005271,-0.490501,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845793,0.634777,0.239816,0.088574,-0.419656,2.849981,...,-0.363747,-0.426301,-0.464667,-0.001433,-0.0027,-0.005271,-0.098101,"[0.0, 0.0, -0.4905010756171413]","[0.0, 0.0, 0.0, 0.0, -0.4905010756171413]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775023,0.639809,0.199889,-0.687843,-0.402874,2.607708,...,-0.180982,-0.315168,-0.402787,-0.001433,-0.0027,-0.005271,0.183075,"[0.0, -0.4905010756171413, -0.09810136648612545]","[0.0, 0.0, 0.0, -0.4905010756171413, -0.098101...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.49..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705031,0.642083,0.161491,1.145565,-0.379061,2.364038,...,-0.133505,-0.264997,-0.368282,-0.001433,-0.0027,-0.005271,-0.037781,"[-0.4905010756171413, -0.09810136648612545, 0....","[0.0, 0.0, -0.4905010756171413, -0.09810136648...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.4905010..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625355,0.643889,0.117219,1.145565,-0.340115,2.088747,...,-0.077878,-0.210807,-0.330137,-0.084172,-0.0027,-0.005271,0.033425,"[-0.09810136648612545, 0.18307478352672518, -0...","[0.0, -0.4905010756171413, -0.0981013664861254...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.490501075617..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,5769,NO 15/9-F-5 AH,2016-08-22,0.268405,-0.880933,-0.806036,-0.814176,1.149547,1.150153,-1.140330,...,-0.693212,-0.702888,-0.716277,-0.692951,-0.704178,-0.725482,-0.681306,"[-0.6834479858098645, -0.6925870358402809, -0....","[-0.6915821804196012, -0.6860958165169114, -0....","[-0.7117159623886966, -0.7111878631895072, -0...."
7983,5769,NO 15/9-F-5 AH,2016-08-23,0.268405,-0.880933,-0.806036,-0.814176,1.159077,1.150153,-1.136215,...,-0.689899,-0.699999,-0.713993,-0.690916,-0.70061,-0.723721,-0.676011,"[-0.6925870358402809, -0.6822891014560878, -0....","[-0.6860958165169114, -0.6834479858098645, -0....","[-0.7111878631895072, -0.7027822842690761, -0...."
7984,5769,NO 15/9-F-5 AH,2016-08-24,0.268405,-0.880933,-0.806036,-0.814176,1.141368,1.150153,-1.137527,...,-0.688108,-0.697863,-0.712046,-0.689665,-0.69802,-0.72149,-0.677250,"[-0.6822891014560878, -0.6813062501687076, -0....","[-0.6834479858098645, -0.6925870358402809, -0....","[-0.7027822842690761, -0.695718957479918, -0.6..."
7985,5769,NO 15/9-F-5 AH,2016-08-25,0.268405,-1.334973,-1.285316,-1.327392,1.140972,1.150153,-1.140776,...,-0.686771,-0.696038,-0.710244,-0.686483,-0.696104,-0.719541,-0.676825,"[-0.6813062501687076, -0.6760105887546141, -0....","[-0.6925870358402809, -0.6822891014560878, -0....","[-0.695718957479918, -0.689095713356751, -0.69..."


In [9]:
for column in multiple_feature_columns:
    try:
        data[column] = data[column].apply(ast.literal_eval)
    except Exception:
        pass

In [10]:
data.isna().any()

well_name                False
WELL_BORE_CODE           False
date                     False
prod_hrs                 False
bhp                      False
bht                      False
dp_tubing                False
AVG_ANNULUS_PRESS        False
AVG_CHOKE_SIZE_P         False
whp                      False
wht                      False
choke_size_percentage    False
oil_vol                  False
gas_vol                  False
water_vol                False
linear_regressor         False
EMA_short                False
EMA_medium               False
EMA_long                 False
Rolling_short            False
Rolling_medium           False
Rolling_long             False
oil_rate                 False
Lag_short                False
Lag_medium               False
Lag_long                 False
dtype: bool

In [11]:
# Get the numerical index of the column 'oil_rate'
column_name = 'oil_rate'
column_index = data[feature_columns].columns.get_loc(column_name)
column_index

19

In [12]:
days_window = 5
predictions_days = 14
training_gap = predictions_days + days_window
batch_size = 96

In [13]:
# sequences = []
# for well_name in data['well_name'].unique():
#     well_data = data[data['well_name'] == well_name].sort_values(by='date').reset_index()
#     for i in range(len(well_data) - training_gap + 1):
#         single_features = np.array(well_data.iloc[i:i + days_window][single_feature_columns])
#         multiple_features = np.array(well_data.iloc[i:i + days_window][multiple_feature_columns])
#         multiple_flattened = []
#         for k in multiple_features:
#            flattened_list = [item for sublist in k for item in sublist]
#            multiple_flattened.append(flattened_list)
#         multiple_flattened = np.array(multiple_flattened)
#         inputs = np.concatenate((single_features, multiple_flattened), axis=1)
#         targets = well_data.iloc[i + days_window:i + days_window + predictions_days][target_column].values
#         inputs_dates = list(well_data.iloc[i:i + days_window]['date'])
#         targets_dates = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['date'])
#         inputs_index = list(well_data.iloc[i:i + days_window]['index'])
#         targets_index = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['index'])
#         sequences.append((well_name, inputs_dates, targets_dates, inputs_index, targets_index, inputs, targets))
#         if len(well_data.iloc[i:i + days_window + predictions_days]['well_name'].unique()) > 1:
#           print('violation')

# dataset = pd.DataFrame(sequences, columns=['well_name', 'inputs_dates', 'targets_dates', 'inputs_index', 'targets_index', 'inputs', 'targets', ])
# dataset.to_pickle(path + "/data/Volve/all_data.pkl")
# dataset

In [14]:
train_size = 0.6
val_size = 0.2
test_size = 0.2

In [15]:
test_size_param1 = test_size
test_size_param2 = val_size / (1 - test_size_param1)

In [16]:
# train_wells = []
# test_wells = []
# for i in well_info.keys():
#     well = dataset[dataset['well_name'] == i]
#     train_set_length = int((train_size + val_size) * len(well))
#     train_wells.append(well[:train_set_length + 1])
#     test_set_length = int(test_size * len(well))
#     test_wells.append(well[-test_set_length:])

# # Flatten the lists of DataFrames
# temp_data = pd.concat(train_wells).reset_index(drop=True)
# data_test = pd.concat(test_wells).reset_index(drop=True)


# data_train, data_val = train_test_split(temp_data, test_size=test_size_param2, stratify=temp_data['well_name'], shuffle=True)

# data_train.reset_index(drop=True, inplace=True)
# data_val.reset_index(drop=True, inplace=True)
# data_test.reset_index(drop=True, inplace=True)

# data_train.to_pickle(path + "/data/Volve/data_train.pkl")
# data_val.to_pickle(path + "/data/Volve/data_val.pkl")
# data_test.to_pickle(path + "/data/Volve/data_test.pkl")

In [17]:
all_data = pd.read_pickle(path + "/data/Volve/all_data.pkl")
data_train = pd.read_pickle(path + "/data/Volve/data_train.pkl")
data_val = pd.read_pickle(path + "/data/Volve/data_val.pkl")
data_test = pd.read_pickle(path + "/data/Volve/data_test.pkl")

In [18]:
data_train.shape, data_val.shape, data_test.shape

((4730, 7), (1577, 7), (1573, 7))

In [19]:
check = True
for i, j in well_info.items():
    if len(all_data[all_data['well_name'] == i]) < 10:
        print("The input/output gap is too high for well", i)
        check = False
if check:
    print("The input/output gap is fine")

The input/output gap is fine


In [20]:
check=True
for i in [data_train, data_val, data_test]:
    for j in well_info.keys():
        if j not in set(list(i['well_name'])):
            print("Some datasets do not have specific well")
            check = False
            break
if check:
    print("All datasets have all wells")

All datasets have all wells


In [21]:
data_train['inputs'][0].shape, data_train['targets'][0].shape

((5, 38), (14, 1))

In [22]:
data_train

,well_name,inputs_dates,targets_dates,inputs_index,targets_index,inputs,targets
0,5351,"[2013-12-01, 2013-12-02, 2013-12-03, 2013-12-0...","[2013-12-06, 2013-12-07, 2013-12-08, 2013-12-0...","[6157, 6158, 6159, 6160, 6161]","[6162, 6163, 6164, 6165, 6166, 6167, 6168, 616...","[[0.2684054414666464, 0.6661194675873554, 0.46...","[[-0.5952839563372336], [-0.5301158502674453],..."
1,5599,"[2014-10-04, 2014-10-05, 2014-10-06, 2014-10-0...","[2014-10-09, 2014-10-10, 2014-10-11, 2014-10-1...","[3742, 3743, 3744, 3745, 3746]","[3747, 3748, 3749, 3750, 3751, 3752, 3753, 375...","[[0.2684054414666464, -1.7396742091238298, -1....","[[-0.8235409678170497], [-0.8219053272417826],..."
2,5351,"[2012-02-17, 2012-02-18, 2012-02-19, 2012-02-2...","[2012-02-22, 2012-02-23, 2012-02-24, 2012-02-2...","[5572, 5573, 5574, 5575, 5576]","[5577, 5578, 5579, 5580, 5581, 5582, 5583, 558...","[[0.2684054414666464, 0.3592646569197392, 0.57...","[[-0.0394163429762045], [-0.0372892767572474],..."
3,7078,"[2014-07-25, 2014-07-26, 2014-07-27, 2014-07-2...","[2014-07-30, 2014-07-31, 2014-08-01, 2014-08-0...","[781, 782, 783, 784, 785]","[786, 787, 788, 789, 790, 791, 792, 793, 794, ...","[[0.2684054414666464, 0.3662688240726383, 0.61...","[[-0.2221166617624337], [-0.0329764666305339],..."
4,5599,"[2013-10-16, 2013-10-17, 2013-10-18, 2013-10-1...","[2013-10-21, 2013-10-22, 2013-10-23, 2013-10-2...","[3416, 3417, 3418, 3419, 3420]","[3421, 3422, 3423, 3424, 3425, 3426, 3427, 342...","[[0.2684054414666464, -1.7396742091238298, -1....","[[-0.7467832161570921], [-0.7441940631388441],..."
...,...,...,...,...,...,...,...
4725,5599,"[2008-05-11, 2008-05-12, 2008-05-13, 2008-05-1...","[2008-05-16, 2008-05-17, 2008-05-18, 2008-05-1...","[1636, 1637, 1638, 1639, 1640]","[1641, 1642, 1643, 1644, 1645, 1646, 1647, 164...","[[0.2684054414666464, 0.7869355293686349, 0.59...","[[2.407758672511925], [2.477086361827733], [2...."
4726,5351,"[2013-08-04, 2013-08-05, 2013-08-06, 2013-08-0...","[2013-08-09, 2013-08-10, 2013-08-11, 2013-08-1...","[6041, 6042, 6043, 6044, 6045]","[6046, 6047, 6048, 6049, 6050, 6051, 6052, 605...","[[0.2684054414666464, 0.6073164371268952, 0.47...","[[-0.3845951820463691], [-0.3835683224923899],..."
4727,5599,"[2014-11-04, 2014-11-05, 2014-11-06, 2014-11-0...","[2014-11-09, 2014-11-10, 2014-11-11, 2014-11-1...","[3773, 3774, 3775, 3776, 3777]","[3778, 3779, 3780, 3781, 3782, 3783, 3784, 378...","[[0.2684054414666464, -1.7396742091238298, -1....","[[-0.816008219517501], [-0.822184046263577], [..."
4728,7078,"[2015-06-19, 2015-06-20, 2015-06-21, 2015-06-2...","[2015-06-24, 2015-06-25, 2015-06-26, 2015-06-2...","[1104, 1105, 1106, 1107, 1108]","[1109, 1110, 1111, 1112, 1113, 1114, 1115, 111...","[[0.2684054414666464, 0.3673082899790074, 0.59...","[[0.2366988563777476], [0.256187183770056], [0..."


In [23]:
# Split sequences into input features and targets
X_train = np.array([i for i in data_train['inputs']])
Y_train = np.array([i for i in data_train['targets']]).squeeze()

X_val = np.array([i for i in data_val['inputs']])
Y_val = np.array([i for i in data_val['targets']]).squeeze()

X_test = np.array([i for i in data_test['inputs']])
Y_test = np.array([i for i in data_test['targets']]).squeeze()

X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape

((4730, 190), (4730, 14), (1577, 190), (1577, 14), (1573, 190), (1573, 14))

In [30]:
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences_df):
        self.sequences_df = sequences_df

    def __len__(self):
        return len(self.sequences_df)

    def __getitem__(self, idx):
        inputs = self.sequences_df.iloc[idx]['inputs']
        targets = self.sequences_df.iloc[idx]['targets']
        inputs, targets = np.array(inputs, np.float32), np.array(targets, np.float32)
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32).squeeze()

train_dataset = TimeSeriesDataset(data_train)
val_dataset = TimeSeriesDataset(data_val)
test_dataset = TimeSeriesDataset(data_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [31]:
class VisualizationTimeSeriesDataset(Dataset):
    def __init__(self, sequences_df):
        self.sequences_df = sequences_df

    def __len__(self):
        return len(self.sequences_df)

    def __getitem__(self, idx):
        inputs = self.sequences_df.iloc[idx]['inputs']
        targets = self.sequences_df.iloc[idx]['targets']
        well_name = self.sequences_df.iloc[idx]['well_name']
        inputs_dates = self.sequences_df.iloc[idx]['inputs_dates']
        targets_dates = self.sequences_df.iloc[idx]['targets_dates']

        return well_name, inputs_dates, targets_dates, torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32).squeeze()

visualization_test_dataset = VisualizationTimeSeriesDataset(data_test)

visualization_test_loader = DataLoader(visualization_test_dataset, batch_size=int(len(test_dataset) / 50), shuffle=True)

In [32]:
def calculate_metrics(y_true, y_pred):
    # Calculate metrics for the predictions
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred, multioutput='uniform_average')
    loss = mean_squared_error(y_true, y_pred)
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'Loss': loss}

def evaluate(mode, model):
    X_modes = {'train': X_train, 'val': X_val, 'test': X_test}
    Y_modes = {'train': Y_train, 'val': Y_val, 'test': Y_test}
    X = X_modes[mode]
    Y = Y_modes[mode].reshape(-1)
    outputs = model.predict(X).reshape(-1)
    # Calculate metrics
    metrics = calculate_metrics(Y, outputs)
    print(f"Loss is {metrics['Loss']:.4f}")
    print(f"MAE is {metrics['MAE']:.4f}")
    print(f"RMSE is {metrics['RMSE']:.4f}")
    print(f"R² is {metrics['R2']:.4f}")
    print("\n")
    return metrics

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    lr = trial.suggest_float('lr', 1e-4, 0.4, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 50)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    xgb = XGBRegressor(output_length= predictions_days, n_estimators=n_estimators, learning_rate=lr, max_depth=max_depth, min_child_weight=min_child_weight, subsample=subsample, colsample_bytree=colsample_bytree)
    model = MultiOutputRegressor(xgb)
    model.fit(X_train, Y_train)
    val_preds = model.predict(X_val)
    val_loss = mean_squared_error(Y_val, val_preds)
    return val_loss

def save_model(model, name=""): # function to save parameters, optimizer and scheduler states
    config_path = path + r'/Models/Global/XGBoost/Model/'
    if name != "":
        name = f'{name}_'
    joblib.dump(model, config_path + f'{name}XGBoost.lib')

def load_model(name=""): 
    config_path = path + r'/Models/Global/XGBoost/Model/'
    if name != "":
        name = f'{name}_'
    model = joblib.load(config_path + f'{name}XGBoost.lib')
    return model

def get_oil_history(date, well_name):
    well = data[data["well_name"] == int(well_name)].reset_index(drop=True)
    well = well[pd.to_datetime(well['date']) < date]
    return well['oil_rate']
    
def label_model_files(old_name="", new_name=""):
    if old_name != "":
        old_name = f'{old_name}_'
    if new_name != "":
        new_name = f'{new_name}_'
    config_path = path + r'/Models/Global/XGBoost/Model/'
    if os.path.exists(config_path + f'{new_name}state_params.pth'):
        try:
            user_choice = int(input("File with new name already exists. Enter 1 if you want to overwrite the files. Otherwise press 2: "))
        except ValueError:
            print("Invalid input. Please enter 1 or 2.")
            return
        if user_choice == 1:
            try:
                os.rename(config_path + f'{old_name}XGBoost.lib', config_path + f'{new_name}XGBoost.lib')
                print("Files have been renamed successfully.")
            except FileNotFoundError:
                print("One or more files not found.")
        elif user_choice == 2:
            print("Operation cancelled by the user.")
        else:
            print("Invalid choice. Operation cancelled.")
    else:
        try:
            os.rename(config_path + f'{old_name}XGBoost.lib', config_path + f'{new_name}XGBoost.lib')
            print("Files have been renamed successfully.")
        except FileNotFoundError:
            print("One or more files not found.")

def predictions_with_history_graphs(model, visualization_loader, path):
    pdf = PdfPages(path + "/predictions_with_history.pdf")
    for well_name, inputs_dates, targets_dates, X_batch, Y_batch in visualization_loader:
        choice = random.randint(0, len(X_batch) - 1)
        X_batch = X_batch[choice].reshape(1, -1)
        Y_batch = Y_batch[choice]
        well_name = well_name[choice]
        inputs_dates = [i[choice] for i in inputs_dates]
        targets_dates = [i[choice] for i in targets_dates]

        history = get_oil_history(pd.to_datetime(targets_dates[0]), well_name)
        prediction = model.predict(X_batch).reshape(-1)
        label = Y_batch
        loss = mean_squared_error(label, prediction)
        rmse = np.sqrt(mean_squared_error(label, prediction))
        # Plotting
        forecast_index = range(len(history) + 1, len(history) + predictions_days + 1)
        plt.figure(figsize=(12, 6))
        plt.plot(range(1, len(history) + 1), history, label='Input Values')
        plt.plot(forecast_index, prediction, label='Forecasted Values')
        plt.plot(forecast_index, label, label='Actual Values')
        # Determine the interval for ticks to have approximately 10 ticks
        tick_interval = max(10, round((len(history) + predictions_days + 1) / 10 / 10) * 10)
        # Generate tick positions
        tick_positions = np.arange(0, len(history) + predictions_days + 1, tick_interval)
        tick_positions[0] = 1  # Start tick at 1 instead of 0

        # Generate tick labels
        tick_labels = tick_positions
        plt.xticks(ticks=tick_positions, labels=tick_labels)
        plt.axvline(x=len(history), color='r', linestyle='--', label='Forecast Start')
        plt.xlabel('Day')
        plt.ylabel('Value')
        plt.title(f'Well: {well_name}, Loss: {loss:.4f}, RMSE = {rmse:.4f}')
        plt.legend()
        plt.grid(True)
        # Save the current figure to the PDF
        pdf.savefig()
        plt.close()
    pdf.close()

def predictions_graphs(model, visualization_loader, path):
    pdf = PdfPages(path + "/predictions.pdf")
    for well_name, inputs_dates, targets_dates, X_batch, Y_batch in visualization_loader:
        choice = random.randint(0, len(X_batch) - 1)
        B, T, C = X_batch.shape
        X_batch = X_batch[choice].reshape(1, -1)
        Y_batch = Y_batch[choice]
        well_name = well_name[choice]
        inputs_dates = [i[choice] for i in inputs_dates]
        targets_dates = [i[choice] for i in targets_dates]

        history = get_oil_history(pd.to_datetime(targets_dates[0]), well_name)
        prediction = model.predict(X_batch).reshape(-1)
        label = Y_batch
        loss = mean_squared_error(label, prediction)
        rmse = np.sqrt(mean_squared_error(label, prediction))

        inputs = X_batch.reshape(1, T, C)[0][:, column_index]
        # Plotting
        forecast_index = range(len(inputs) + 1, len(inputs) + predictions_days + 1)
        plt.figure(figsize=(12, 6))
        plt.plot(range(1, len(inputs) + 1), inputs, marker='o', label='Input Values')
        plt.plot(forecast_index, prediction, marker='o', label='Forecasted Values')
        plt.plot(forecast_index, label, marker='o', label='Actual Values')
        plt.xticks(ticks=np.arange(1, len(inputs) + predictions_days + 1), labels=np.arange(1, len(inputs) + predictions_days + 1))
        plt.axvline(x=len(inputs), color='r', linestyle='--', label='Forecast Start')
        plt.xlabel('Day')
        plt.ylabel('Value')
        plt.title(f'Well: {well_name}, Loss: {loss:.4f}, RMSE = {rmse:.4f}')
        plt.legend()
        plt.grid(True)
        # Save the current figure to the PDF
        pdf.savefig()
        plt.close()
    pdf.close()

def all_predictions_graphs(model, data, data_test, path):
    well_dataloaders = {}
    pdf = PdfPages(path + "/all_predictions.pdf")
    for well_name in well_info.keys():
        well = data[data['well_name'] == well_name].reset_index(drop=True)
        fig, ax = plt.subplots(figsize=(15, 6))
        ax.plot(well.index, well['oil_rate'], label='Well Production')
        ax.axvline(x=int((1 - test_size) * len(well) - days_window - 2), color='r', linestyle='--',)
        well_dataset = VisualizationTimeSeriesDataset(data_test[data_test['well_name'] == well_name].reset_index(drop=True))
        well_dataloaders[well_name] = DataLoader(well_dataset, batch_size=1, shuffle=False)
        total_loss = 0
        total_rmse = 0
        for i, (_, inputs_dates, targets_dates, X_batch, Y_batch) in enumerate(well_dataloaders[well_name]):
            B, T, C = X_batch.shape
            inputs_dates = np.array(inputs_dates).T[0]
            targets_dates = np.array(targets_dates).T[0]
            history = get_oil_history(pd.to_datetime(targets_dates[0]), well_name)
            X_batch = X_batch.reshape(B, T*C)
            prediction = model.predict(X_batch).reshape(-1)
            label = Y_batch.reshape(-1)
            loss = mean_squared_error(label, prediction)
            rmse = np.sqrt(mean_squared_error(label, prediction))
            total_loss += loss
            total_rmse += rmse
            forecast_index = range(len(history) + 1, len(history) + predictions_days + 1)
            ax.plot(forecast_index, prediction, label='Forecasted Values', color='y')
            if i == 0:
                ax.legend()
        avg_loss = total_loss / len(well_dataloaders[well_name]) 
        avg_rmse = total_rmse / len(well_dataloaders[well_name]) 
        ax.set_xlabel('Day')
        ax.set_ylabel('Value')
        ax.set_title(f'Well: {well_name}, Loss: {avg_loss:.4f}, RMSE = {avg_rmse:.4f}')
        ax.grid(True)
        pdf.savefig(fig)
        plt.close()
    pdf.close()

def all_predictions_with_gaps_graphs(model, data, data_test, path):
    well_plots = {}
    well_dataloaders = {}   
    pdf = PdfPages(path + "/all_predictions_with_gaps.pdf")
    for well_name in well_info.keys():
        well = data[data['well_name'] == well_name].reset_index(drop=True)
        fig, ax = plt.subplots(figsize=(15, 6))
        ax.plot(well.index, well['oil_rate'], label='Well Production')
        ax.axvline(x=int((1 - test_size) * len(well) - days_window - 2), color='r', linestyle='--',)
        well_dataset = VisualizationTimeSeriesDataset(data_test[data_test['well_name'] == well_name].reset_index(drop=True))
        well_dataloaders[well_name] = DataLoader(well_dataset, batch_size=1, shuffle=False)
        total_loss = 0
        total_rmse = 0
        for i, (_, inputs_dates, targets_dates, X_batch, Y_batch) in enumerate(well_dataloaders[well_name]):
            if i % predictions_days == 0 or i == len(well_dataloaders[well_name]) - 1:
                B, T, C = X_batch.shape
                inputs_dates = np.array(inputs_dates).T[0]
                targets_dates = np.array(targets_dates).T[0]
                history = get_oil_history(pd.to_datetime(targets_dates[0]), well_name)
                X_batch = X_batch.reshape(B, T*C)
                prediction = model.predict(X_batch).reshape(-1)
                label = Y_batch.reshape(-1)
                loss = mean_squared_error(label, prediction)
                rmse = np.sqrt(mean_squared_error(label, prediction))
                total_loss += loss
                total_rmse += rmse
                forecast_index = range(len(history) + 1, len(history) + predictions_days + 1)
                ax.plot(forecast_index, prediction, label='Forecasted Values', color='y')
                if i == 0:
                    ax.legend()
        avg_loss = total_loss / len(well_dataloaders[well_name]) 
        avg_rmse = total_rmse / len(well_dataloaders[well_name]) 
        ax.set_xlabel('Day')
        ax.set_ylabel('Value')
        ax.set_title(f'Well: {well_name}, Loss: {avg_loss:.4f}, RMSE = {avg_rmse:.4f}')
        ax.grid(True)
        pdf.savefig(fig)
        plt.close()
    pdf.close()

In [92]:
%%capture
# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
best_params = study.best_params
n_estimators = best_params['n_estimators']
lr = best_params['lr']
max_depth = best_params['max_depth']
min_child_weight = best_params['min_child_weight']
subsample = best_params['subsample']
colsample_bytree = best_params['colsample_bytree']

[I 2024-07-02 14:53:20,150] A new study created in memory with name: no-name-0dbfd818-6c65-4881-a15f-f42b6644bd43
[I 2024-07-02 14:55:04,966] Trial 0 finished with value: 0.029965498943533227 and parameters: {'n_estimators': 806, 'lr': 0.0055242708973689885, 'max_depth': 23, 'min_child_weight': 34, 'subsample': 0.9276311307669547, 'colsample_bytree': 0.8895332353188312}. Best is trial 0 with value: 0.029965498943533227.
[I 2024-07-02 14:56:33,209] Trial 1 finished with value: 0.6925375320870095 and parameters: {'n_estimators': 559, 'lr': 0.0004101348293390399, 'max_depth': 14, 'min_child_weight': 3, 'subsample': 0.7218004340618807, 'colsample_bytree': 0.6437446549615271}. Best is trial 0 with value: 0.029965498943533227.
[I 2024-07-02 14:58:39,561] Trial 2 finished with value: 0.02355662488090007 and parameters: {'n_estimators': 752, 'lr': 0.021883664323391568, 'max_depth': 30, 'min_child_weight': 16, 'subsample': 0.5216476775900352, 'colsample_bytree': 0.8877023913280608}. Best is tri

In [93]:
print('Best hyperparameters:', best_params)

Best hyperparameters: {'n_estimators': 594, 'lr': 0.045570993763519016, 'max_depth': 17, 'min_child_weight': 12, 'subsample': 0.7433625471337193, 'colsample_bytree': 0.5992551725398402}


In [33]:
n_estimators = 594
lr = 0.045570993763519016 
max_depth = 17
min_child_weight = 12
subsample = 0.7433625471337193 
colsample_bytree = 0.5992551725398402

In [34]:
# Create and configure the XGBoost regressor with hyperparameters
xgb = XGBRegressor(n_estimators=n_estimators, learning_rate=lr, max_depth=max_depth, min_child_weight=min_child_weight, subsample=subsample, colsample_bytree=colsample_bytree) 

# Wrap the XGBoost regressor in a MultiOutputRegressor
model = MultiOutputRegressor(xgb)
model.fit(X_train, Y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.5992551725398402,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.045570993763519016,
                                            max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=17,
                                            max_leaves=None,
                                            min_child_weight=12, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=594, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))

In [35]:
train_metrics = evaluate('train', model)
val_sizemetrics = evaluate('val', model)
test_metrics = evaluate('test', model)

Loss is 0.0003
MAE is 0.0079
RMSE is 0.0181
R² is 0.9997


Loss is 0.0407
MAE is 0.0920
RMSE is 0.2016
R² is 0.9627


Loss is 0.0048
MAE is 0.0511
RMSE is 0.0696
R² is 0.7992




In [35]:
save_model(model, "current")

In [36]:
label_model_files(old_name='current', new_name='best')

Files have been renamed successfully.


In [37]:
model = load_model("best")

In [91]:
predictions_with_history_graphs(model, visualization_test_loader, path + "/Models/Global/XGBoost/Graphs")
predictions_graphs(model, visualization_test_loader, path + "/Models/Global/XGBoost/Graphs")
all_predictions_graphs(model, data, data_test, path + "/Models/Global/XGBoost/Graphs")
all_predictions_with_gaps_graphs(model, data, data_test, path + "/Models/Global/XGBoost/Graphs")
test_metrics = evaluate('test', model)

Loss is 0.0043
MAE is 0.0481
RMSE is 0.0656
R² is 0.8202


